In [ ]:
### Cell 1: Load Necessary Libraries & Read Data
import pandas as pd
import matplotlib.pyplot as plt

# Load Excel file
file_path = "shadowhistory.xlsx"  # Ensure the file is in the same directory
xls = pd.ExcelFile(file_path)
df = pd.read_excel(xls, sheet_name=xls.sheet_names[0])

# Convert columns to appropriate types
df["Open Time"] = pd.to_datetime(df["Open Time"], format="%Y.%m.%d %H:%M:%S")
df["Close Time"] = pd.to_datetime(df["Close Time"], format="%Y.%m.%d %H:%M:%S")
df["Profit"] = pd.to_numeric(df["Profit"], errors="coerce")

# Sort data by Item (currency pair) and Open Time
df = df.sort_values(by=["Item", "Open Time"]).reset_index(drop=True)

### Cell 2: Function to Count Levels in Smart Averaging Cycles
def count_levels(trade_data):
    level_counts = {}
    ongoing_cycle = []
    
    for i in range(len(trade_data)):
        lot_size = trade_data.loc[i, "Size"]
        
        if not ongoing_cycle or lot_size == 0.01:
            if ongoing_cycle:
                max_level = len(ongoing_cycle)
                level_counts[max_level] = level_counts.get(max_level, 0) + 1
            ongoing_cycle = [lot_size]
        else:
            ongoing_cycle.append(lot_size)
    
    if ongoing_cycle:
        max_level = len(ongoing_cycle)
        level_counts[max_level] = level_counts.get(max_level, 0) + 1
    
    return level_counts

### Cell 3: Process Trade Data for Each Currency Pair
trade_levels = {}
for pair in df["Item"].unique():
    pair_data = df[df["Item"] == pair].copy()
    
    buy_trades = pair_data[pair_data["Type"] == "buy"].reset_index(drop=True)
    sell_trades = pair_data[pair_data["Type"] == "sell"].reset_index(drop=True)
    
    buy_levels = count_levels(buy_trades)
    sell_levels = count_levels(sell_trades)
    
    trade_levels[pair] = {"Buy": buy_levels, "Sell": sell_levels}

### Cell 4: Compute Overall Statistics
overall_levels = {"Buy": {}, "Sell": {}}
for pair in trade_levels:
    for trade_type in ["Buy", "Sell"]:
        for level, count in trade_levels[pair][trade_type].items():
            overall_levels[trade_type][level] = overall_levels[trade_type].get(level, 0) + count
trade_levels["Overall"] = overall_levels

### Cell 5: Extract Additional Insights
def extract_trade_insights(df):
    insights = {}
    
    df["Duration"] = (df["Close Time"] - df["Open Time"]).dt.total_seconds() / 60  # Duration in minutes
    
    avg_duration = df.groupby("Item")["Duration"].mean()
    avg_profit = df.groupby("Item")["Profit"].mean()
    
    insights["Avg Duration per Pair"] = avg_duration.to_dict()
    insights["Avg Profit per Pair"] = avg_profit.to_dict()
    
    return insights

trade_insights = extract_trade_insights(df)

### Cell 6: Display Results
for pair, data in trade_levels.items():
    print(f"\n{pair.upper()} Trade Levels:")
    for trade_type in ["Buy", "Sell"]:
        print(f"\n{trade_type} Trades:")
        for level, count in sorted(data[trade_type].items()):
            print(f"Level {level}: {count} times")

# Display overall insights
print("\nOverall Trading Insights:")
print(trade_insights)
